# Noether Checks

Check that the BdNMC runs on Noether performed nominally

In [2]:
import uproot3 as up3
import numpy as np
import pandas as pd
import os

* Checks for missing .root files
* Checks for missing .log / .out / .err / .sub files
    * Check .sub file matches .root file params
* Reads all the .log / .out / .err files for each job, checks for errors
* Reads the root files, checks
    * N_chi > 10_000
    * Ma / mchi = dt ratio of file
    * Anything else


### Check For Missing .ROOT Files

In [12]:
# get a list of the card files that I used to generate the data
card_list_files = []
for file in os.listdir("../Noether"):
    if "pi0" in file:
        card_list_files.append(file)
    elif "eta" in file:
        card_list_files.append(file)
    else:
        continue

print(card_list_files)

# read these files and look for a root file in data/root/bdnmc that has the same name
for file in card_list_files:
    with open(f"../Noether/{file}") as f:
        lines = f.readlines()
        for line in lines:
            root_file_name = line[:-5] + ".root"
            if not(os.listdir(f"../data/root/BdNMC/{root_file_name}")):
                print(root_file_name, " not found!")
            
            # could have a look inside the root file here too
            #f = up3.open(f"../data/root/BdNMC/{root_file_name}")
            # f.close()

['card_list_fermion_pi0_0.6.txt', 'card_list_scalar_eta_0.6.txt', 'card_list_fermion_eta_0.33.txt', 'card_list_fermion_eta_0.6.txt', 'card_list_scalar_pi0_0.33.txt', 'card_list_scalar_pi0_2.0.txt', 'card_list_scalar_eta_0.33.txt', 'card_list_scalar_eta_2.0.txt', 'card_list_fermion_pi0_2.0.txt', 'card_list_scalar_pi0_0.6.txt', 'card_list_fermion_pi0_0.33.txt', 'card_list_fermion_eta_2.0.txt']


FileNotFoundError: [Errno 2] No such file or directory: '../data/root/BdNMC/fermion/pi0_fermion_ma_0.01_dt_0.6.root'

### Check the log/out/err/sub files

In [4]:
PATH = "../data/condor_out/"

In [5]:
f_sub = []
f_out = []
f_err = []
f_log = []
# loop through all files in the directory
for file in os.listdir(PATH):
    if file.endswith(".sub"):
        f_sub.append(file)
    elif file.endswith(".out"):
        f_out.append(file)
    elif file.endswith(".err"):
        f_err.append(file)
    elif file.endswith(".log"):
        f_log.append(file)
    else:
        print("Unknown file type: ", file)

FileNotFoundError: [Errno 2] No such file or directory: '../data/condor_out/'